In [1]:
!wget https://github.com/mozilla/geckodriver/releases/download/v0.33.0/geckodriver-v0.33.0-linux-aarch64.tar.gz
!tar -xvzf geckodriver*
!chmod +x geckodriver
!export PATH=$PATH:.

--2023-05-31 18:14:11--  https://github.com/mozilla/geckodriver/releases/download/v0.33.0/geckodriver-v0.33.0-linux-aarch64.tar.gz
^C

gzip: stdin: not in gzip format
tar: Child returned status 1
tar: Error is not recoverable: exiting now


In [1]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
from yahooquery import Ticker
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib as plt
import math
from pandas.tseries.offsets import DateOffset
from pandas.tseries.holiday import USFederalHolidayCalendar
from datetime import datetime
from copy import deepcopy

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

import warnings

warnings.filterwarnings("ignore")

In [2]:
#https://stackoverflow.com/questions/45448994/wait-page-to-load-before-getting-data-with-requests-get-in-python-3

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
import sys  

options = webdriver.FirefoxOptions()
options.add_argument('--headless')
s = Service('./geckodriver')
#user_agent = 'Mozilla/5.0 (X11; Linux x86_64 AppleWebKit/537.36 (KHTML, like GECKO) Chrome/33.0.1750.517 Safari/537.36'
#options.add_argument('user-agent={0}'.format(user_agent))
# executable_path param is not needed if you updated PATH
#driver = webdriver.Chrome(executable_path='/Users/andrew/Documents/workspace/instabot/chromedriver/chromedriver')
browser = webdriver.Firefox(options=options, service=s)
ticker = 'MSFT'
url = f"https://www.nasdaq.com/market-activity/stocks/{ticker}/earnings"
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, features="html.parser")
browser.quit()

In [3]:
all_earnings_tags = soup.find_all('td', class_='earnings-surprise__table-cell')
earnings_dates = []
for tag in all_earnings_tags:
    content = tag.get_text()
    if '/' in content:
        earnings_dates.append(content)
print(earnings_dates)

['04/25/2023', '01/24/2023', '10/25/2022', '07/26/2022']


In [4]:
options = webdriver.FirefoxOptions()
options.add_argument('--headless')

#user_agent = 'Mozilla/5.0 (X11; Linux x86_64 AppleWebKit/537.36 (KHTML, like GECKO) Chrome/33.0.1750.517 Safari/537.36'
#options.add_argument('user-agent={0}'.format(user_agent))
# executable_path param is not needed if you updated PATH
#driver = webdriver.Chrome(executable_path='/Users/andrew/Documents/workspace/instabot/chromedriver/chromedriver')

browser = webdriver.Firefox(options=options, executable_path='./geckodriver')
ticker = 'ALSN'
url = f"https://marketchameleon.com/Overview/{ticker}/Earnings/Earnings-Dates/"
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, features="html.parser")
browser.quit()

In [5]:
html_ = str(soup)
key_ = html_.find("earnings on")
aoi = html_[key_:key_+100]
date = " ".join(aoi.split(" ")[2:5])
is_before = "BMC" in aoi
is_after = "AMC" in aoi

In [6]:
earn_date = datetime.strptime(date, '%B %d, %Y')

tickers = Ticker(ticker, asynchronous=True)

# Default period = ytd, interval = 1d

events = tickers.calendar_events

earnings_hist = tickers.earnings_trend

# if the call is in the morning, previous day close - earnings day open
# if the call is in the afternoon, earnings day close - next day open
if is_before:
    after_date = deepcopy(earn_date) + dt.timedelta(days = 1)
    before_date = deepcopy(earn_date) - dt.timedelta(days = 1)
elif is_after:
    after_date = deepcopy(earn_date) + dt.timedelta(days = 2)
    before_date = deepcopy(earn_date)
else:
    raise KeyError

if before_date.weekday() in [5,6]:
    before_date = before_date - dt.timedelta(days=before_date.weekday()-4)

if after_date.weekday() in [5,6]:
    after_date = after_date + dt.timedelta(days = 7-after_date.weekday())

# check for federal holidays
USFederalHolidayCalendar

df = tickers.history(start=before_date, end=after_date, period='1d')

(df.iloc[1]["open"] - df.iloc[0]["close"]) / df.iloc[0]["close"]

0.04979163084373924

In [7]:
after_date.weekday()
after_date in USFederalHolidayCalendar().holidays(start='2014-01-01', end='2030-12-31').to_pydatetime()

False

In [8]:
str(before_date), before_date.weekday(), str(after_date), after_date.weekday() 

('2023-04-27 00:00:00', 3, '2023-05-01 00:00:00', 0)

# Looping over bunches of stocks

In [9]:
ticker = 'UNH'

def get_surprise_binary(ticker):
    temp = None  # Ensure this variable is cleared
    temp = Ticker(ticker)
    temp_df = None  # Ensure this variable is cleared
    temp_df = temp.earning_history.reset_index().copy()
    if temp_df.shape[0] == 0:
        return 0
    try:
        temp_df["date"] = pd.to_datetime(temp_df["quarter"], format="%Y-%m-%d")
    except:
        print(ticker, temp_df["quarter"])
        return 0
    # Create new features
    temp_df["surpriseBinary"] = temp_df["epsDifference"].apply(
        lambda x: 1 if float(x) > 0 else 0
    )
    
    temp_df.sort_values(by=['quarter'],inplace=True,ascending=False)
    return ticker, temp_df.surpriseBinary.iloc[0]

get_surprise_binary("UNH")

('UNH', 1)

In [10]:
def get_price_mov(ticker):
    print(ticker)
    options = webdriver.FirefoxOptions()
    options.add_argument('--headless')
    s = Service('./geckodriver')

    browser = webdriver.Firefox(options=options, service=s)
    url = f"https://marketchameleon.com/Overview/{ticker}/Earnings/Earnings-Dates/"
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    browser.quit()
    
    html_ = str(soup)
    key_ = html_.find("earnings on")
    aoi = html_[key_:key_+100]
    date = " ".join(aoi.split(" ")[2:5])
    print(date, aoi)
    is_before = "BMO" in aoi
    is_after = "AMC" in aoi
    
    earn_date = datetime.strptime(date, '%B %d, %Y')

    tickers = Ticker(ticker, asynchronous=True)
    
    # Default period = ytd, interval = 1d
    
    events = tickers.calendar_events
    
    earnings_hist = tickers.earnings_trend
    
    # if the call is in the morning, previous day close - earnings day open
    # if the call is in the afternoon, earnings day close - next day open
    if is_before:
        after_date = deepcopy(earn_date) + dt.timedelta(days = 1)
        before_date = deepcopy(earn_date) - dt.timedelta(days = 1)
    else:# is_after:
        after_date = deepcopy(earn_date) + dt.timedelta(days = 2)
        before_date = deepcopy(earn_date)
    # else:
    #    raise KeyError
    
    if before_date.weekday() in [5,6]:
        before_date = before_date - dt.timedelta(days=before_date.weekday()-4)
    
    if after_date.weekday() in [5,6]:
        after_date = after_date + dt.timedelta(days = 7-after_date.weekday())
    
    # check for federal holidays
    # USFederalHolidayCalendar # TODO
    try:
        df = tickers.history(start=before_date, end=after_date, period='1d')
        return ticker, (df.iloc[1]["open"] - df.iloc[0]["close"]) / df.iloc[0]["close"]
    except:
        return ticker, np.nan

In [14]:
from mlresearch.utils import parallel_loop
from esurprise.data import DataLoader

tickers = DataLoader("snp500").fetch_symbols()["symbol"].tolist()
surprise_binary = parallel_loop(get_surprise_binary, tickers, n_jobs=-1, progress_bar=True)
price_mov = parallel_loop(get_price_mov, tickers, n_jobs=-1, progress_bar=True)

LiveError: Only one live display may be active at once

In [13]:
res = pd.DataFrame(price_mov, columns=["ticker", "price_mov"]).set_index("ticker")
res2 = pd.DataFrame([i for i in surprise_binary if type(i)!=int], columns=["ticker", "surp_bin"]).set_index("ticker")
res_total = pd.concat([res, res2], axis=1).dropna()
res_total[res_total["surp_bin"]>0].median()

NameError: name 'price_mov' is not defined